In [5]:
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import requests
import json
import re

result = list()

def getSongs():
    """song_ids와 ranks에서 id에 matching되는 정보와 rank를 json 파일로 반환"""
    # result = list()
    for rank, song_id in zip(ranks, song_ids):
        sleep(5)
        print(song_id)

        req = requests.get('http://www.melon.com/song/detail.htm?songId=' + song_id, headers = header)
        html = req.text
        soup = BeautifulSoup(html, "lxml")

        title = soup.find(attrs={"class": "song_name"}).text.replace('곡명', '')
        if '19금' in title:
            title = title.replace('19금', '')
        title = re.sub('^\s*|\s+$','', title)
        artist = soup.find(attrs={"class": "artist_name"}).text
        album = soup.select('#downloadfrm > div > div > div.entry > div.meta > dl > dd')[0].text
        genre = soup.select('#downloadfrm > div > div > div.entry > div.meta > dl > dd')[2].text
        release = soup.select('#downloadfrm > div > div > div.entry > div.meta > dl > dd')[1].text

        try:
            lyric = soup.find('div', attrs={"class": "lyric"}).get_text()
            lyric = re.sub('<[^>]*>|\s|\[|\]', ' ', lyric)
            lyric = re.sub('^\s*|\s+$', '', lyric)
        except:
            print("week_xpath not found")
            continue

        result.append({
            'year': re.sub('[^0-9]', '', year.text),
            'month': re.sub('[^0-9]', '', month.text),
            'week': week.text, 
            'rank': rank.text,
            'title': title,
            'artist': artist,
            'album': album,
            'genre': genre,
            'id': song_id,                
            'release': release
            'lyric': lyric
            })
        print("차트 연도:", year.text)
        print("차트 월간:", month.text)
        print("차트 주간:", week.text)
        print("순위:", rank.text)
        print("곡 id:", song_id)
        print("제목:", title)
        print("아티스트:", artist)
        print("*_*_*_*_*_*_*_*_*_*_*__*_*_*")
                
                
# acess the website    
header = {'User-Agent': ''}
d = webdriver.Chrome('/Users/jeongseohyeong/Dropbox/Yonsei/Summer 18/ESC 여름 스터디/파이썬 스터디/7주차/chromedriver')
d.implicitly_wait(3)
d.get('http://www.melon.com/chart/index.htm')
d.get("http://www.melon.com/chart/search/index.htm")

# //: 현재 node로부터 문서상 모든 node 조회
# *: 매칭되는 모든 element node
# //*[@id="d_chart_search"]: id="blahblah" 속성값을 가지는 모든 element 선택
d.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[1]/a').click()
# h4[1]: 주간차트 / h4[2]: 월간차트 / h4[3]: 연도차트

# 연대선택
age_xpath = '//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[1]/span/label'
age = d.find_element_by_xpath(age_xpath)
age.click()

for i in [7]: # 6: 2013, 5: 2014 순서대로 (옛날부터) 7: 2012, 8: 2011
    # 년도선택
    year_xpath = '//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[' + str(i) + ']/span/label'
    year = d.find_element_by_xpath(year_xpath)
    year.click()
    
    for i in [1,2,3,4,5,6,7,8,9,10,11,12]:
        # 월선택
        month_xpath = '//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[' + str(i) + ']/span/label'
        month = d.find_element_by_xpath(month_xpath)
        month.click()
        
        for i in [1,2,3,4,5]:
            # 주간선택
            try:
                week_xpath = '//*[@id="d_chart_search"]/div/div/div[4]/div[1]/ul/li[' + str(i) + ']/span/label'
                week = d.find_element_by_xpath(week_xpath)
                week.click()
            except:
                print("week_xpath not found")
                continue
            
            # 차트 가요 선택: 11년도 12년도 국내종합 분류가 없음
            classCd = d.find_element_by_xpath('//label[@for = "gnr_1"]')
            if '가요' not in classCd.text:
                classCd = d.find_element_by_xpath('//label[@for = "gnr_2"]')
            classCd.click()
            d.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span').click()
            sleep(10)
            
            # get song ids and ranks for rank 1~50: song50_ids & ranks50
            song_ids = d.find_elements_by_xpath('//*[@id="lst50"]/td[4]/div/a')
            song_ids = [re.sub('[^0-9]', '', song_id.get_attribute("href")) for song_id in song_ids]
            ranks = d.find_elements_by_xpath('//*[@id="lst50"]/td[2]/div/span[1]')
            
            getSongs()
            
            # access next page
            next = d.find_element_by_xpath('//*[@id="frm"]/div[2]/span/a') 
            next.click()
            # get song ids and ranks for rank 51~100: song100_ids & ranks100
            song_ids = d.find_elements_by_xpath('//*[@id="lst100"]/td[4]/div/a')
            song_ids = [re.sub('[^0-9]', '', song_id.get_attribute("href")) for song_id in song_ids]
            ranks = d.find_elements_by_xpath('//*[@id="lst100"]/td[2]/div/span[1]')

            getSongs()

d.quit()
#print("THIS IS THE LIST, RESULT")
#print(result)

with open('/Users/jeongseohyeong/Dropbox/Yonsei/Fall_18/Data Science/project/melonchart_11_12.json', 'a', encoding='utf-8') as f:
    j = json.dumps(result)
    f.write(j)
    f.close()

3625504
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 1
곡 id: 3625504
제목: 너랑 나
아티스트: 아이유
앨범: Last Fantasy
장르: Dance
발매일: 2011.11.29
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3630755
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 2
곡 id: 3630755
제목: Trouble Maker
아티스트: 트러블메이커
앨범: Trouble Maker
장르: Dance
발매일: 2011.12.01
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3654576
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 3
곡 id: 3654576
제목: 우리 사랑했잖아
아티스트: 다비치
앨범: 우리 사랑했잖아
장르: Ballad
발매일: 2011.12.23
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3663978
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 4
곡 id: 3663978
제목: Lovey-Dovey
아티스트: 티아라
앨범: Funky Town
장르: Dance
발매일: 2012.01.03
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3656320
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 5
곡 id: 3656320
제목: 속상해서 (술 한잔 해요 Part.2)
아티스트: 지아
앨범: 속상해서
장르: Ballad
발매일: 2011.12.27
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3607859
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 6
곡 id: 3607859
제목: Cry Cry
아티스트: 티아라
앨범: Black Eyes
장르: Dance
발매일: 2011.11.11
*_*_*_*_*_*_*_*_*_*_*__*_*_*

3573175
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 48
곡 id: 3573175
제목: 연애시대 (Feat. Ra.D) (Narr. 한효주)
아티스트: 이승기
앨범: 연애시대
장르: Ballad
발매일: 2011.10.14
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3600222
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 49
곡 id: 3600222
제목: G.N.O.
아티스트: 원더걸스
앨범: Wonder World
장르: Dance
발매일: 2011.11.07
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3648682
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 50
곡 id: 3648682
제목: I Believe
아티스트: 이승철
앨범: I Believe
장르: Ballad
발매일: 2011.12.19
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3588757
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 51
곡 id: 3588757
제목: Get It In (Feat. Tiger JK, 정인) (Korean Ver.)
아티스트: 윤미래
앨범: Get It In (The Creators Project)
장르: Rap / Hip-hop
발매일: 2011.10.27
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3536905
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 52
곡 id: 3536905
제목: 안녕이라고 말하지마
아티스트: 다비치
앨범: Love Delight
장르: Ballad
발매일: 2011.08.29
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3635328
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 53
곡 id: 3635328
제목: 하얀 고백 (Lat

3604726
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 94
곡 id: 3604726
제목: 그댄 정말 모를거예요
아티스트: 제이세라
앨범: Tears Of Crystal
장르: Ballad
발매일: 2011.11.10
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3664709
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 95
곡 id: 3664709
제목: 느낌Good (Feat. 장재인)
아티스트: 윤종신
앨범: 2012 月刊 尹鍾信 January
장르: Ballad
발매일: 2012.01.04
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3600225
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 96
곡 id: 3600225
제목: Me, in
아티스트: 원더걸스
앨범: Wonder World
장르: Dance
발매일: 2011.11.07
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3667495
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 97
곡 id: 3667495
제목: 예쁜게 다니
아티스트: 살찐고양이
앨범: 예쁜게 다니
장르: Dance
발매일: 2012.01.06
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3651645
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 98
곡 id: 3651645
제목: Sexy Doll (With 코타 & 주비 Of Sunnyhill / Rap MYK)
아티스트: 클래지
앨범: Sexy Doll
장르: Electronica
발매일: 2011.12.21
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3663196
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.01~01.07
순위: 99
곡 id: 3663196
제목: P.S. I Love You (박정현)
아티스트:

3652920
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.08~01.14
순위: 41
곡 id: 3652920
제목: 이별사랑
아티스트: 이영현
앨범: 이별사랑
장르: Ballad
발매일: 2011.12.22
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3641162
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.08~01.14
순위: 42
곡 id: 3641162
제목: 미행
아티스트: 먼데이 키즈 (Monday Kiz)
앨범: The Ballad
장르: Ballad
발매일: 2011.12.12
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3589429
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.08~01.14
순위: 43
곡 id: 3589429
제목: 친구잖아
아티스트: 이승기
앨범: Tonight
장르: Ballad
발매일: 2011.10.27
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3661428
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.08~01.14
순위: 44
곡 id: 3661428
제목: Sun & Star (Feat. 임정희)
아티스트: 주석
앨범: Sun & Star (Feat. 임정희)
장르: Rap / Hip-hop
발매일: 2011.12.30
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3549031
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.08~01.14
순위: 45
곡 id: 3549031
제목: Hello
아티스트: 허각
앨범: LIKE 1st MINI ALBUM `First Story`
장르: Ballad
발매일: 2011.09.16
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3593454
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.08~01.14
순위: 46
곡 id: 3593454
제목: 막걸리나
아티스트: 버스커 버스커
앨범: 슈퍼스타K 3 Top11 Part 5
장르: Ro

3667616
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.08~01.14
순위: 87
곡 id: 3667616
제목: 그녀가 온다
아티스트: 카오스
앨범: 그녀가 온다
장르: Dance
발매일: 2012.01.06
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3664709
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.08~01.14
순위: 88
곡 id: 3664709
제목: 느낌Good (Feat. 장재인)
아티스트: 윤종신
앨범: 2012 月刊 尹鍾信 January
장르: Ballad
발매일: 2012.01.04
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3638253
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.08~01.14
순위: 89
곡 id: 3638253
제목: 겨울밤
아티스트: 장재인
앨범: 겨울밤
장르: Ballad
발매일: 2011.12.08
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3570032
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.08~01.14
순위: 90
곡 id: 3570032
제목: Open Arms (Journey) (Feat. 크리스티나)
아티스트: 울랄라세션
앨범: 슈퍼스타K 3 Top11 Part 2
장르: Ballad
발매일: 2011.10.10
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3623496
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.08~01.14
순위: 91
곡 id: 3623496
제목: 행복했다...안녕
아티스트: 김연우
앨범: Mr. Big
장르: Ballad
발매일: 2011.11.24
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3667472
차트 연도: 2012년
차트 월간: 01월
차트 주간: 01.08~01.14
순위: 92
곡 id: 3667472
제목: 이별을 앞둔 모든 이들에게 (Feat. 정세영 Of Take Out)
아티스트: PS영준
앨범: 이별을 앞

...

3986114
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.10~12.16
순위: 96
곡 id: 3986114
제목: 난 널 잊었어 (Feat. 에일리)
아티스트: 주석
앨범: 5 Point 5
장르: Rap / Hip-hop
발매일: 2012.12.11
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3986055
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.10~12.16
순위: 97
곡 id: 3986055
제목: Geto Christmas (Feat. Mini)
아티스트: MC 스나이퍼
앨범: Geto Christmas
장르: Rap / Hip-hop
발매일: 2012.12.11
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3853980
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.10~12.16
순위: 98
곡 id: 3853980
제목: 어땠을까 (Feat. 박정현)
아티스트: 싸이 (PSY)
앨범: 싸이6甲 Part.1
장르: Rap / Hip-hop
발매일: 2012.07.15
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3957742
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.10~12.16
순위: 99
곡 id: 3957742
제목: 연극이 끝난 후
아티스트: 딕펑스
앨범: 슈퍼스타K4 Top 12 Part.5
장르: Dance
발매일: 2012.11.12
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3984702
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.10~12.16
순위: 100
곡 id: 3984702
제목: 첫사랑을 찾습니다
아티스트: 서영은
앨범: 첫사랑을 찾습니다
장르: Ballad
발매일: 2012.12.10
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3968836
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.17~12.23
순위: 1
곡 id: 3968836
제목: 되돌리다
아티스트: 이승기

3170749
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.17~12.23
순위: 43
곡 id: 3170749
제목: Officially Missing You
아티스트: 긱스 (Geeks)
앨범: Officially Missing You
장르: Rap / Hip-hop
발매일: 2011.03.09
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3922586
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.17~12.23
순위: 44
곡 id: 3922586
제목: 춥다 (Feat. 이하이)
아티스트: 에픽하이 (EPIK HIGH)
앨범: 99
장르: Rap / Hip-hop
발매일: 2012.10.19
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3884575
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.17~12.23
순위: 45
곡 id: 3884575
제목: I Need You
아티스트: 허각
앨범: I Need You
장르: Ballad
발매일: 2012.08.21
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3957743
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.17~12.23
순위: 46
곡 id: 3957743
제목: 응급실
아티스트: 정준영
앨범: 슈퍼스타K4 Top 12 Part.5
장르: Ballad
발매일: 2012.11.12
*_*_*_*_*_*_*_*_*_*_*__*_*_*
1589050
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.17~12.23
순위: 47
곡 id: 1589050
제목: 사랑이 다른 사랑으로 잊혀지네
아티스트: 하림
앨범: Whistle In A Maze
장르: Ballad
발매일: 2004.04.09
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3948917
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.17~12.23
순위: 48
곡 id: 3948917
제목: If You Love Me (

3990649
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.17~12.23
순위: 90
곡 id: 3990649
제목: 광화문 연가
아티스트: 정동하
앨범: 불후의 명곡 - 전설을 노래하다 (겨울특집편)
장르: Ballad
발매일: 2012.12.15
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3981147
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.17~12.23
순위: 91
곡 id: 3981147
제목: 나 같은 놈
아티스트: 백퍼센트
앨범: 나 같은 놈
장르: Ballad
발매일: 2012.12.07
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3983288
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.17~12.23
순위: 92
곡 id: 3983288
제목: My Sweetie
아티스트: 길미
앨범: My Sweetie
장르: Rap / Hip-hop
발매일: 2012.12.07
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3987961
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.17~12.23
순위: 93
곡 id: 3987961
제목: 칵테일 파티 이펙트 (With 최진이 Of 럼블피쉬)
아티스트: 소울다이브
앨범: 칵테일 파티 이펙트 (With 최진이 Of 럼블피쉬)
장르: Rap / Hip-hop
발매일: 2012.12.13
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3964485
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.17~12.23
순위: 94
곡 id: 3964485
제목: 떠나지마
아티스트: 딕펑스
앨범: 슈퍼스타K4 Top 12 Part.6
장르: Rock
발매일: 2012.11.19
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3964472
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.17~12.23
순위: 95
곡 id: 3964472
제목: 24시간후
아티스트: 2BIC(투빅)
앨범: 

3928294
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.24~12.30
순위: 37
곡 id: 3928294
제목: 남자 없이 잘 살아
아티스트: 미쓰에이
앨범: Independent Women pt.Ⅲ
장르: Dance
발매일: 2012.10.15
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3991037
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.24~12.30
순위: 38
곡 id: 3991037
제목: My Grown Up Christmas List (Christmas Wish)
아티스트: 에일리
앨범: Hitman Project #1 : A Tribute To The Hitman, David Foster
장르: Ballad
발매일: 2012.12.18
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3928194
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.24~12.30
순위: 39
곡 id: 3928194
제목: 말리꽃 (원곡가수 이승철)
아티스트: 이정
앨범: `나는 가수다 2` 10월 B조 경연
장르: Ballad
발매일: 2012.10.14
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3928713
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.24~12.30
순위: 40
곡 id: 3928713
제목: 닐리리맘보
아티스트: 블락비 (Block B)
앨범: BLOCKBUSTER
장르: Dance
발매일: 2012.10.17
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3978258
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.24~12.30
순위: 41
곡 id: 3978258
제목: 백야
아티스트: 넬 (NELL)
앨범: Holding Onto Gravity
장르: Rock
발매일: 2012.12.03
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3170749
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.24~

3996321
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.24~12.30
순위: 83
곡 id: 3996321
제목: 슬픈 인연 (원곡가수 나미)
아티스트: 이은미
앨범: `나는 가수다 2` 2012 가왕전 세미파이널
장르: Ballad
발매일: 2012.12.23
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3053259
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.24~12.30
순위: 84
곡 id: 3053259
제목: 미리 메리 크리스마스 (Feat. 천둥 Of MBLAQ)
아티스트: 아이유
앨범: Real
장르: Dance
발매일: 2010.12.09
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3966937
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.24~12.30
순위: 85
곡 id: 3966937
제목: 사랑해 봤니
아티스트: 이정
앨범: Thank You
장르: Ballad
발매일: 2012.11.21
*_*_*_*_*_*_*_*_*_*_*__*_*_*
4000627
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.24~12.30
순위: 86
곡 id: 4000627
제목: 아프지마 (Feat. 현아 Of 4Minute)
아티스트: 이루
앨범: 아프지마
장르: Dance
발매일: 2012.12.28
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3955584
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.24~12.30
순위: 87
곡 id: 3955584
제목: 내 맘은 죽어가요 (Feat. Speed)
아티스트: 더 씨야
앨범: Good To Seeya
장르: Ballad
발매일: 2012.11.12
*_*_*_*_*_*_*_*_*_*_*__*_*_*
3983343
차트 연도: 2012년
차트 월간: 12월
차트 주간: 12.24~12.30
순위: 88
곡 id: 3983343
제목: 12월의 이야기
아티스트: 스윗소로우
앨범: Sweet 